In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import seaborn as sns
import cftime

# ```xarray``` is your best friend

### opening data

### plotting spatial data

# Some commonly used operations

### Regridding data

### grid-wise correlation

### grid-to-point correlation

### spatial average

### remove seasonal cycle

### detrending

### resampling

# Conceptual points

Generate some random timeseries here? E.g., Hasselman's climate model.
- Make histogram of random 40-year segment
- plot ensemble spread
Next: try on pre-industrial control!

# PI control example
- white-noise process vs. PI control
- histogram of 40-year trends

In [ ]:
## set plotting style
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## initialize random number generator
rng = np.random.default_rng()


def get_T_bar(t, trend, trend_type="exp"):
    """Get 'background' temperature"""

    ## get final and initial times
    tf = t[-1]
    ti = t[0]

    ## get "background" temperature
    if trend_type == "exp":
        a = 3e-2
        b = trend * (tf - ti) * np.exp(-a * (tf - ti))
        T_bar = b * np.exp(a * (t - ti))

    elif trend_type == "linear":
        T_bar = (t - t[0]) * trend

    else:
        print("Not a valid trend type")
        T_bar = 0.0

    return T_bar


def markov_simulation(
    ti, tf, dt=1 / 365.25, g=-0.3, n=0.3, n_members=1, trend=0, trend_type="exp"
):
    """Minimal version of the 'stochastic climate model' studied
    by Hasselman et al (1976) and Frankignoul and Hasselmann (1977).
    (See Eqn 3.6 in Frankignoul and Hasselman, 1976). The damping rate
    used here is much lower than in the paper, for illustration purposes.
    Args:
        - ti: number representing initial time (units: years)
        - tf: number representing final time (units: years)
        - dt: timestep (units: years)
        - g: damping rate (equivalent to lambda in the paper; units: 1/year)
        - n: noise amplitude (units: K / year^{1/2})
        - n_members: number of ensemble members
        - trend: calculated increase in "background" T, with units of 1/year
            (based on (T[tf]-T[ti]) / (tf-ti)
        - trend_type: one of "exp" (exponential) or "linear"
    """

    ## Get timesteps and dimensions for output
    t = np.arange(ti, tf, dt)
    nt = len(t)

    ## Create empty arrays arrays to hold simulation output
    T = np.zeros([n_members, nt])

    ## get "background" temperature
    T_bar = get_T_bar(t, trend, trend_type)[None, :]

    for i, t_ in tqdm(enumerate(t[:-1])):
        dW = np.sqrt(dt) * rng.normal(size=n_members)
        dT = g * (T[:, i] - T_bar[:, i]) * dt + n * dW
        T[:, i + 1] = T[:, i] + dT

    ##  put in xarray
    # time_idx = pd.date_range(start=f"{ti}-01-01", periods=nt, freq="1D")
    time_idx = xr.cftime_range(start=cftime.datetime(ti, 1, 1), periods=nt, freq="1D")
    e_member_idx = pd.Index(np.arange(1, n_members + 1), name="ensemble_member")

    T = xr.DataArray(
        T,
        dims=["ensemble_member", "time"],
        coords={"ensemble_member": e_member_idx, "time": time_idx},
    )

    ## resample to Annual
    T = T.resample({"time": "YS"}).mean()

    ## change time coordinate to year
    year = T.time.dt.year.values
    T = T.rename({"time": "year"})
    T["year"] = year

    return T


n_members = 50
tf = 2006
T_PI = markov_simulation(ti=850, tf=tf, n_members=n_members, trend=0)
T_warming = markov_simulation(ti=1850, tf=tf, n_members=n_members, trend=0.01)

## for convenience, get subset of pre-industrial control which overlaps with warming
T_PI_hist = T_PI.sel(year=T_warming.year)

## Plot ensemble means

In [ ]:
def plot_ensemble_spread(ax, T, color, label=None):
    """plot mean and +/- 1 standard dev. of ensemble on
    given ax object."""

    ## compute stats
    mean = T.mean("ensemble_member")
    std = T.std("ensemble_member")

    ## plot mean
    mean_plot = ax.plot(mean.year, mean, label=label, color=color)

    ## plot spread
    ax.plot(mean.year, mean + std, lw=0.5, c=mean_plot[0].get_color())
    ax.plot(mean.year, mean - std, lw=0.5, c=mean_plot[0].get_color())

    return


## Plot ensemble stats
fig, ax = plt.subplots(figsize=(4, 3))

## plot data
plot_ensemble_spread(ax, T_PI_hist, color="black", label="warming")
plot_ensemble_spread(ax, T_warming, color="red", label="P.I. control")

## label axes
ax.set_xlabel("Year")
ax.set_ylabel(r"SST anomaly ($^{\circ}C$)")
ax.legend(prop={"size": 10})
ax.set_title("Ensemble results")

plt.show()

## plot a random sample from each
idx = rng.choice(T_PI.ensemble_member)
fig, ax = plt.subplots(figsize=(4, 3))

ax.plot(
    T_PI_hist.year,
    T_PI_hist.sel(ensemble_member=idx),
    color="black",
    label="P.I. control",
)
ax.plot(
    T_warming.year, T_warming.sel(ensemble_member=idx), color="red", label="warming"
)

## label axes
ax.set_xlabel("Year")
ax.set_ylabel(r"SST anomaly ($^{\circ}C$)")
ax.legend(prop={"size": 10})
ax.set_title("random ensemble member")

plt.show()

# SSP/RCP example
- validate against reanalysis in historical period (1950-2000)
    - e.g., $\mu$, $\sigma$, seasonal cycle, spectrum
- what are future projections?